<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [410]:
# ##### PORTFOLIO INVESTOR CODE #####
#
# Code to upload data in Google Sheets to support investiments decisions.
# The lists with the tickers and other informations should be filled directly in the code where there are INSERT OR ADJUST HERE lines.
# IFIX historic series csv file with time desired (2y, 3y, etc.) from Investing.com should be downloaded to a Google Drive directory which is in the path in code - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data .
# FUNDS EXPLORER - Copy and paste date from https://www.fundsexplorer.com.br/ranking in a Google Drive directory excel file which is in the path in code. It is necessary to SELECT ALL COLUMNS in Funds Explorer website.
# FUNDAMENTUS - Copy and paste date from https://www.fundamentus.com.br/buscaavancada.php in a Google Drive directory excel file which is in the path in code. To generate the website page DO NOT SELECT ANY FILTER in the search.
# Before to run the code it is necessary to make these activities.  Check the code below until the ENDING POINT TO MANUAL ACTIVITIES.
# Verifiy data on thes files in case of code running problems.

In [411]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
import os
import datetime as dt
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [412]:
# Stock tickers - Insert here# Real Estate Invesments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IBOV + 25 assets in alphabetical order to easly adjust google docs spreadsheet
tickers = ['^BVSP','ABEV3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CMIN3.SA','CPFE3.SA','CPLE6.SA','ELET6.SA','EMBR3.SA','ITUB4.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','MBRF3.SA','LEVE3.SA','LREN3.SA','PETR4.SA','POMO4.SA','RECV3.SA','SBSP3.SA','TGMA3.SA','VALE3.SA','VIVA3.SA','VIVT3.SA','WEGE3.SA']

In [413]:
# Real Estate Investments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IFIX + 20 assets in alphabetical order to easly adjust google docs spreadsheet
# IFIX here only to a space in dataframe, yahoo finance return IFIX.SA maximum 5 days. IFIX should be calculated downloading an Investing.com csv file history and it must be uploaded in personal Google Drive. This code read it.
tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MXRF11.SA','MFII11.SA','SAPI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']

In [414]:
# US Stocks tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 20 assets in alphabetical order to easly adjust google docs spreadsheet
tickersus = ['^GSPC','USDBRL=X','AAPL','AIG','BAC','RIO','DHI','EXC','KMB','KO','LOPE','LYB','MGA','MSFT','MSTR','NUE','NVDA','TGT','TMUS','UPS','UNH','XOM']

In [415]:
# US ETFs tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickereus = ['^GSPC','USDBRL=X','FBTC','JEPI','HACK', 'IVV','SCHD','SOXX','SPY','TLT','SMH','TFLO']

In [416]:
# Portfolio tickers - INSERT OR ADJUST HERE
tickerport = ['^BVSP','USDBRL=X','BBSE3.SA','BTLG11.SA','CPFE3.SA','ELET6.SA','FBTC','HSML11.SA','HTMX11.SA','IVVB11.SA','KLBN11.SA','PETR4.SA','SAPI11.SA','SCHD','TFLO','TGAR11.SA','TRXF11.SA','VALE3.SA']
# Portfolio tickers weight - INSERT OR ADJUST HERE IN THE SAME ORDER!
weightport = [0.0, 0.0, 0.000, 0.000, 4.157, 8.367, 4.010, 3.081, 5.964, 8.889, 0.000, 11.148, 7.653, 29.928, 3.830, 4.296, 5.059, 3.618]
# Portfolio tickers expected returns - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) MUST always be zero!
expretport = [0.0, 0.0, 21.4, 7.8, 9.6, -9.1, 18.0, 11.3, 21.2, 20.0, 16.4, 34.6, 16.3, 18.0, 12.0, 23.0, 19.2, 5.9]

In [417]:
# Risk free rate in percentage - INSERT OR ADJUST HERE
riskfree = 13.88

In [418]:
# Target Sharpe in percentage of Sharpe Estimated Maximum - INSERT OR ADJUST HERE
target_per = 75.00

In [419]:
# Portfolio dataframe creation
portfolio = pd.DataFrame({'Ticker': tickerport, 'W': weightport, 'RetE%':expretport})

In [420]:
# Excluding .SA, renaming ^BVSP to IBOV and USDBRL=X to USDBRL
portfolio['Ticker'] = portfolio['Ticker'].str.replace('.SA', '', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('^BVSP', 'IBOV', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(portfolio)

In [421]:
# Load tickers history prices in a dataframe considering a certain period of time - ADJUST HERE, default 1 year (1y)
# Sometimes some ticker has problems in yahoo finance. If it happens, close the session and try again. Or change the ticker, because problem in one ticker will cause problem in all code running.
# Check success download completed to all dataframes, otherwise the code will broke in next lines.
dfs = yf.download(tickers, period='2y', auto_adjust=True)['Close']
dfr = yf.download(tickerr, period='2y', auto_adjust=True)['Close']
dfsus = yf.download(tickersus, period='2y', auto_adjust=True)['Close']
dfeus = yf.download(tickereus, period='2y', auto_adjust=True)['Close']
dfport = yf.download(tickerport, period='2y', auto_adjust=True)['Close']
# Remove timezone from index
dfs.index = pd.to_datetime(dfs.index).tz_localize(None)
dfr.index = pd.to_datetime(dfr.index).tz_localize(None)
dfsus.index = pd.to_datetime(dfsus.index).tz_localize(None)
dfeus.index = pd.to_datetime(dfeus.index).tz_localize(None)
dfport.index = pd.to_datetime(dfport.index).tz_localize(None)
# display(dfs)
# display(dfr)
# display(dfeus)
# display(dfport)

[*********************100%***********************]  26 of 26 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  18 of 18 completed


In [422]:
# Convert US assets in BRL values, it should be adjusted according the assets in portfolio - ADJUST HERE
dfport['SCHD'] = dfport['SCHD'] * dfport['USDBRL=X']
dfport['TFLO'] = dfport['TFLO'] * dfport['USDBRL=X']
dfport['FBTC'] = dfport['FBTC'] * dfport['USDBRL=X']
# display(dfport)

In [423]:
# IFIX historic series from Investing.com to be appended in real state dataframe dfr - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data - MAKE THIS ACTIVITY
# Download the file from site and copy to your google drive. Rename de file as history.csv. Adjust the path below in " ifixfile = .... " command line according your file location.
# Google Drive mounth
drive.mount('/content/drive', force_remount=True)
# File path on Google Drive - Download the file and upload to Financas folder in Google Drive. Rename the path according file name uploaded.
ifixfile = '/content/drive/MyDrive/Financas/history.csv'
# File csv to dataframe converting quote to float
ifix = pd.read_csv(ifixfile, thousands = '.', decimal = ',', dtype = {'Último':np.float64})
# Excluding and rename columns
ifix = ifix.drop(columns=['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'])
ifix = ifix.rename(columns={'Data': 'Date', 'Último': 'IFIX.SA'})
# Date format in Date column
ifix['Date'] = pd.to_datetime(ifix['Date'], format='%d%m%Y', errors='coerce')
ifix.set_index('Date', inplace=True)
# Solve eventual duplicated registers, grouped by mean
ifix = ifix.groupby(level=0).mean()
# display(ifix)

Mounted at /content/drive


In [424]:
# Go to Funds Explorer Table in https://www.fundsexplorer.com.br/ranking  - MAKE THIS ACTIVITY
# Manually select all columns in web page and paste all data AS VALUES ONLY in a NEW Excel File sheet.
# Save the file with the name fundsexplorer.xlsx in a folder called Financas (or in another preferable path, adjusting the path below)
fundsexplorerfile = '/content/drive/MyDrive/Financas/fundsexplorer.xlsx'
# Excel file reading
fundsexplorer = pd.read_excel(fundsexplorerfile)
#display(fundsexplorer)
# The sequence of FUNDS EXPLORER ROUTINE, see at the end of the code.

In [425]:
# Go to Fundamentus website https://www.fundamentus.com.br/buscaavancada.php  - MAKE THIS ACTIVITY
# Press the Search (Buscar) button. A searching with all filters in blank.
# Manually select all columns in web page and paste all data AS VALUES ONLY in a NEW Excel File sheet.
# Save the file with the name fundsexplorer.xlsx in a folder called Financas (or in another preferable path, adjusting the path below)
fundamentusfile = '/content/drive/MyDrive/Financas/fundamentus.xlsx'
# Excel file reading
fundamentus = pd.read_excel(fundamentusfile)
#display(fundsexplorer)
# The sequence of FUNDS EXPLORER ROUTINE, see at the end of the code.

In [426]:
# ENDING POINT TO MANUAL ACTIVITIES. The code should run after this point. Permission to Google Drive access can be required during code running.

In [427]:
# Replace dfr dataframe by ifix values by index key (date)
dfr.update(ifix)
# display(dfr)

In [428]:
# Excluding .SA, renaming ^BVSP to IBOV, ^GSPC to SP500
dfs.columns = [col.replace('.SA', '') for col in dfs.columns]
dfs.columns = [col.replace('^BVSP', 'IBOV') for col in dfs.columns]
dfr.columns = [col.replace('.SA', '') for col in dfr.columns]
dfsus.columns = [col.replace('^GSPC', 'SP500') for col in dfsus.columns]
dfsus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfsus.columns]
dfeus.columns = [col.replace('^GSPC', 'SP500') for col in dfeus.columns]
dfeus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfeus.columns]
dfport.columns = [col.replace('.SA', '') for col in dfport.columns]
dfport.columns = [col.replace('^BVSP', 'IBOV') for col in dfport.columns]
dfport.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfport.columns]

In [429]:
# Exclude NaNs, in the first row and in the dfr Date registers that not included in IFIX excel file.
dfs.dropna(inplace=True)
dfr.dropna(inplace=True)
dfsus.dropna(inplace=True)
dfeus.dropna(inplace=True)
dfport.dropna(inplace=True)
# Other conformations such as ascending order and the market indexes in the first column
dfs = dfs[sorted(dfs.columns)]
dfr = dfr[sorted(dfr.columns)]
dfsus = dfsus[sorted(dfsus.columns)]
dfus = dfeus[sorted(dfeus.columns)]
dfport = dfport[sorted(dfport.columns)]
dfs = dfs[['IBOV'] + [col for col in dfs.columns if col != 'IBOV']]
dfr = dfr[['IFIX'] + [col for col in dfr.columns if col != 'IFIX']]
dfsus = dfsus[['SP500', 'USDBRL'] + [col for col in dfsus.columns if col not in ['SP500', 'USDBRL']]]
dfeus = dfeus[['SP500', 'USDBRL'] + [col for col in dfeus.columns if col not in ['SP500', 'USDBRL']]]
dfport = dfport[['IBOV', 'USDBRL'] + [col for col in dfport.columns if col not in ['IBOV', 'USDBRL']]]
# Here dataframes should be ready for calculations. It will be made and uploaded in dfxvar dataframes later.
# display(dfs)
# display(dfr)
# display(dfsus)
# display(dfeus)
# display(dfport)

In [430]:
# Calculate daily variation
dfsvar = dfs.pct_change()
dfrvar = dfr.pct_change()
dfsusvar = dfsus.pct_change()
dfeusvar = dfeus.pct_change()
dfportvar = dfport.pct_change()
# Excluding MaNs first line
dfsvar.dropna(inplace=True)
dfrvar.dropna(inplace=True)
dfsusvar.dropna(inplace=True)
dfeusvar.dropna(inplace=True)
dfportvar.dropna(inplace=True)
# display(dfsvar)
# display(dfrvar)
# display(dfsusvar)
# display(dfeusvar)
# display(dfportvar)

In [431]:
# Market Percentage Historic Return calculation and column add in output dataframes
stockvar = pd.DataFrame(dfsvar.mean()*252*100).rename(columns={0: 'RetH%'})
stockvar.index.name = 'Ticker'
stockvar['RetH%'] = stockvar['RetH%'].round(1)
realstatevar = pd.DataFrame(dfrvar.mean()*252*100).rename(columns={0: 'RetH%'})
realstatevar.index.name = 'Ticker'
realstatevar['RetH%'] = realstatevar['RetH%'].round(1)
stockusvar = pd.DataFrame(dfsusvar.mean()*252*100).rename(columns={0: 'RetH%'})
stockusvar.index.name = 'Ticker'
stockusvar['RetH%'] = stockusvar['RetH%'].round(1)
etfusvar = pd.DataFrame(dfeusvar.mean()*252*100).rename(columns={0: 'RetH%'})
etfusvar.index.name = 'Ticker'
etfusvar['RetH%'] = etfusvar['RetH%'].round(1)
portvar = pd.DataFrame(dfportvar.mean()*252*100).rename(columns={0: 'RetH%'})
portvar.index.name = 'Ticker'
portvar['RetH%'] = portvar['RetH%'].round(1)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [432]:
# Market return variance calculation
vars = dfsvar.var()*252
varr = dfrvar.var()*252
varsus = dfsusvar.var()*252
vareus = dfeusvar.var()*252
varport = dfportvar.var()*252
# display(vars)
# display(varr)
# display(varsus)
# display(vareus)
# display(varport)

In [433]:
# Market risk calculation, in percentage (%). Add column in output dataframes
stockvar['Risk%'] = dfsvar.std()*np.sqrt(252)*100
stockvar['Risk%'] = stockvar['Risk%'].round(0)
realstatevar['Risk%'] = dfrvar.std()*np.sqrt(252)*100
realstatevar['Risk%'] = realstatevar['Risk%'].round(0)
stockusvar['Risk%'] = dfsusvar.std()*np.sqrt(252)*100
stockusvar['Risk%'] = stockusvar['Risk%'].round(0)
etfusvar['Risk%'] = dfeusvar.std()*np.sqrt(252)*100
etfusvar['Risk%'] = etfusvar['Risk%'].round(0)
portvar['Risk%'] = dfportvar.std()*np.sqrt(252)*100
portvar['Risk%'] = portvar['Risk%'].round(0)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [434]:
# Covariance calculation
covs = dfsvar.cov()*252
covr = dfrvar.cov()*252
covsus = dfsusvar.cov()*252
coveus = dfeusvar.cov()*252
covport = dfportvar.cov()*252
# display(covs)
# display(covr)
# display(covsus)
# display(coveus)
# display(covport)

In [435]:
# Beta calculation
betas = covs['IBOV']/vars['IBOV']
betas = betas.round(3)
betas.name = 'Beta'
betar = covr['IFIX']/varr['IFIX']
betar = betar.round(3)
betar.name = 'Beta'
betasus = covsus['SP500']/varsus['SP500']
betasus = betasus.round(3)
betasus.name = 'Beta'
betaeus = coveus['SP500']/vareus['SP500']
betaeus = betaeus.round(3)
betaeus.name = 'Beta'
betaport = covport['IBOV']/varport['IBOV']
betaport = betaport.round(3)
betaport.name = 'Beta'
# display(betas)
# display(betar)
# display(betasus)
# display(betaeus)
# display(betaport)

In [436]:
# Adding Beta to column output dataframes
stockvar['Beta'] = betas
realstatevar['Beta'] = betar
stockusvar['Beta'] = betasus
etfusvar['Beta'] = betaeus
portvar['Beta'] = betaport
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [437]:
# Adding Min to column output dataframes
stockvar['Min'] = dfs.min()
stockvar['Min'] = stockvar['Min'].round(2)
realstatevar['Min'] = dfr.min()
realstatevar['Min'] = realstatevar['Min'].round(2)
stockusvar['Min'] = dfsus.min()
stockusvar['Min'] = stockusvar['Min'].round(2)
etfusvar['Min'] = dfeus.min()
etfusvar['Min'] = etfusvar['Min'].round(2)
portvar['Min'] = dfport.min()
portvar['Min'] = portvar['Min'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [438]:
# Adding Max to column output dataframes
stockvar['Max'] = dfs.max()
stockvar['Max'] = stockvar['Max'].round(2)
realstatevar['Max'] = dfr.max()
realstatevar['Max'] = realstatevar['Max'].round(2)
stockusvar['Max'] = dfsus.max()
stockusvar['Max'] = stockusvar['Max'].round(2)
etfusvar['Max'] = dfeus.max()
etfusvar['Max'] = etfusvar['Max'].round(2)
portvar['Max'] = dfport.max()
portvar['Max'] = portvar['Max'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [439]:
# Organizing columns order and making Index column as index of dataframes
stockvar = stockvar.reset_index()
realstatevar = realstatevar.reset_index()
stockusvar = stockusvar.reset_index()
etfusvar = etfusvar.reset_index()
portvar = portvar.reset_index()
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [440]:
# FUNCTIONS TO CALCULATE PORTFOLIO ERFORMANCE, MAXIMUM SHARPE AND TARGET SHARPE
# --- Exemplo de uso ---
# Supondo que você tem df_returns (DataFrame) com retornos log ou simples, calculados diariamente/mensalmente:
# mu = df_returns.mean().values           # retorno médio por ativo (periodicidade compatível)
# cov = df_returns.cov().values           # matriz de covariância
# risk_free = 0.0                         # ajuste conforme sua periodicidade (ex: taxa anual/252 para dia)
#
# res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
# print("Max Sharpe:", portfolio_performance(w_max, mu, cov, risk_free)[2], "pesos:", w_max)
#
# target = 1.2
# res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree)
# print("Resultado objetivo:", info)
# print("Pesos:", w_tgt)

In [441]:
# Portfolio dataframe assembling
portfolio = pd.merge(portfolio, portvar, on='Ticker', how='inner')
# portfolio = portfolio[sorted(portfolio.columns)]
# portfolio = portfolio[['IBOV', 'USDBRL'] + [col for col in portfolio.columns if col not in ['IBOV', 'USDBRL']]]
# firstsnames = ['IBOV','USDBRL']
firstsnames = ['IBOV', 'USDBRL']
portfolio.iloc[2:] = portfolio[~portfolio['Ticker'].isin(firstsnames)].sort_values(by='Ticker').values
# display(portfolio)

In [442]:
# Portfolio PERFORMANCE calculation function
def portfolio_performance(weights, mu, cov, riskfree):
    """
    Retorna (retorno_portfolio, volatilidade_portfolio, sharpe_portfolio)
    expected_returns: vetor de retornos esperados por ativo (numpy array)
    cov: matriz de covariância (numpy array)
    weights: vetor de pesos
    riskfree: taxa livre de risco (mesma periodicidade)
    """
    w = np.array(weights)
    ret = float(np.dot(w, mu))
    vol = float(np.sqrt(w.T @ cov @ w))
    if vol == 0:
        sharpe = 0.0
    else:
        sharpe = (ret - riskfree) / vol
    return ret, vol, sharpe

In [443]:
riskfree = riskfree / 100 # Risk Free is indicated in percentage

In [444]:
weights = [values / 100 for values in weightport] # Expected Returns are indicated in percentage

In [445]:
# Convert covariance dataframe in numpy matrix
cov = covport.values

In [446]:
# Extracting expected returns
mu = portfolio['RetE%'].values / 100

In [447]:
# Porfolio Expected Return, Expected Risk and Expected Sharpe calculations
portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp = portfolio_performance(weights, mu, cov, riskfree)
# print (portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp)

In [448]:
# Portfolio adding columns with Total Return, Total Risk and Sharpe values in first register(in the same line of IBOV value index). Other registers being filled with zero.
portfolio['RetET%'] = [portfolioretexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetET%'] = portfolio['RetET%'].round(1)
portfolio['RiskET%'] = [portfolioriskexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RiskET%'] = portfolio['RiskET%'].round(0)
portfolio['SharpeE'] = [portfoliosharpeexp] + [0] * (len(portfolio) - 1)
portfolio['SharpeE'] = portfolio['SharpeE'].round(3)

In [449]:
# Extracting real returns
mu = portfolio['RetH%'].values / 100

In [450]:
# Porfolio Real (historic) Return, Risk and Sharpe calculations
portfoliorettotal, portfoliorisktotal, portfoliosharpe = portfolio_performance(weights, mu, cov, riskfree)
# print (portfoliorettotal, portfoliorisktotal, portfoliosharpe)

In [451]:
portfolio['RetHT%'] = [portfoliorettotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetHT%'] = portfolio['RetHT%'].round(1)
portfolio['SharpeH'] = [portfoliosharpe] + [0] * (len(portfolio) - 1)
portfolio['SharpeH'] = portfolio['SharpeH'].round(3)

In [452]:
# Calculate Total Beta and add in IBOV line
portfolio.at[0, 'Beta'] = (portfolio['W'] / 100 * portfolio['Beta']).sum()
portfolio['Beta'] = portfolio['Beta'].round(3)
# display(portfolio)

In [453]:
# Portfolio MAXIMUM SHARPE calculation function
def max_sharpe_weights(mu, cov, risk_free, bounds=None, constraints=None):
    """
    Encontra os pesos que maximizam o Sharpe (usando minimize sobre -sharpe).
    bounds: lista de tuplas (min, max) por ativo. padrão: (0,1) para todos.
    constraints: lista de constraint dicts para scipy.optimize.minimize (opcional).
    Retorna o dicionário de resultado e os pesos ótimos.
    """
    n = len(mu)
    if bounds is None:
        bounds = tuple((0.0, 1.0) for _ in range(n))
    # restrição: soma dos pesos = 1
    cons = [{'type':'eq', 'fun': lambda w: np.sum(w) - 1.0}]
    if constraints:
        cons.extend(constraints)
    # inicialização: pesos iguais
    x0 = np.ones(n) / n

    def neg_sharpe(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return -s

    res = minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res, res.x if res.success else None

In [454]:
# Extracting expected returns to calculte Sharpe Maximum
mu = portfolio['RetE%'].values / 100

In [455]:
res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
print("Max Sharpe:", portfolio_performance(w_max, mu, cov, riskfree)[2], "pesos:", w_max)
# print("Ret Max Sharpe:", w_max)

Max Sharpe: 1.1948707441298272 pesos: [0.00000000e+00 0.00000000e+00 1.91754849e-01 0.00000000e+00
 1.39852718e-16 3.69627496e-16 3.63271288e-16 0.00000000e+00
 1.02906221e-01 1.09072310e-01 2.56672305e-16 2.17081291e-01
 1.64273095e-02 2.54837261e-16 4.10305466e-16 7.41408602e-02
 2.88617159e-01 3.52477684e-16]


In [456]:
#  Add Maximum Sharpe ticker weights to portfolio dataframe
portfolio['ShMaxE-W'] = w_max.round(3) * 100
# display(portfolio)

In [457]:
# Add Maximum Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShMaxE-W'] = round(portfolio_performance(w_max, mu, cov, riskfree)[2], 3)
# display(portfolio)

In [458]:
# Add Sharpe Maximum Return column and the value in the first line
portfolio['ShMaxRetE%'] = [portfolio_performance(w_max, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShMaxRetE%'] = portfolio['ShMaxRetE%'].round(1)
# display(portfolio)

In [459]:
# Portfolio TARGET SHARPE calculation function
def weights_for_target_sharpe(mu, cov, target_sharpe, riskfree, bounds=None, tol=1e-6, maxiter=1000):
    """
    Encontra pesos que aproximem um Sharpe alvo, minimizando o erro quadrático entre Sharpe(weights) e target_sharpe.
    Retorna o resultado da otimização e os pesos.
    Observação: A solução pode ser inviável exatamente; a rotina busca o vetor que minimize o erro.
    """
    n = len(mu)
    if bounds is None:
        bounds = tuple((0.0, 1.0) for _ in range(n))

    # restrição: soma dos pesos = 1
    cons = [{'type':'eq', 'fun': lambda w: np.sum(w) - 1.0}]

    # inicialização: use o max-sharpe como chute inicial, se possível
    try:
        res_max, w0 = max_sharpe_weights(mu, cov, riskfree, bounds=bounds)
        if w0 is None:
            x0 = np.ones(n) / n
        else:
            x0 = w0
    except Exception:
        x0 = np.ones(n) / n

    def sharpe_err_sq(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return (s - target_sharpe)**2

    opts = {'maxiter': maxiter, 'ftol': tol}
    res = minimize(sharpe_err_sq, x0, method='SLSQP', bounds=bounds, constraints=cons, options=opts)

    # informação extra: distância do objetivo e performance
    if res.success:
        ret, vol, s = portfolio_performance(res.x, mu, cov, riskfree)
        info = {'target_sharpe': target_sharpe, 'achieved_sharpe': s, 'ret': ret, 'vol': vol}
    else:
        info = {'message': res.message}

    return res, res.x if res.success else None, info


In [460]:
# TARGET SHARPE definition
# Target the average between Estimated Sharpe and Maximum Estimated Sharpe
# target = portfolio.at[0, 'SharpeE'] + ((portfolio.at[0, 'ShMaxE-W'] - portfolio.at[0, 'SharpeE']) / 2)
# Target as percentage of Maximum Estimated Sharpe
target = portfolio.at[0, 'ShMaxE-W'] * target_per / 100
print("Target:", target)
res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree)
# print("Resultado objetivo:", info)
# print("Pesos:", w_tgt)
# print("Achieved Sharpe:", info['achieved_sharpe'])

Target: 0.89625


In [461]:
#  Add Target Sharpe ticker weights to portfolio dataframe
portfolio['ShTg-W'] = (w_tgt * 100).round(3)
# Add Target (Achieved) Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShTg-W'] = round(info['achieved_sharpe'], 3)
# Add Target Maximum Return column and the value in the first line
portfolio['ShTgRetE%'] = [portfolio_performance(w_tgt, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShTgRetE%'] = portfolio['ShTgRetE%'].round(1)
display(portfolio)

,Ticker,W,RetE%,RetH%,Risk%,Beta,Min,Max,RetET%,RiskET%,SharpeE,RetHT%,SharpeH,ShMaxE-W,ShMaxRetE%,ShTg-W,ShTgRetE%
0,IBOV,0.000,0.0,8.7,14.0,0.367,118533.00,149540.00,16.9,10.0,0.289,14.5,0.057,1.195,23.5,0.896,20.8
1,USDBRL,0.000,0.0,6.3,13.0,-0.006,4.86,6.30,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0
2,BBSE3,0.000,21.4,11.6,17.0,0.447,27.38,40.42,0.0,0.0,0.000,0.0,0.000,19.200,0.0,17.641,0.0
3,BTLG11,0.000,7.8,10.6,12.0,-0.017,79.11,103.90,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0
4,CPFE3,4.157,9.6,6.9,20.0,0.641,31.00,41.68,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.324,0.0
5,ELET6,8.367,-9.1,22.0,22.0,1.019,35.38,59.39,0.0,0.0,0.000,0.0,0.000,0.000,0.0,6.002,0.0
6,FBTC,4.010,18.0,68.2,53.0,0.530,171.30,583.82,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.001,0.0
7,HSML11,3.081,11.3,5.4,15.0,0.145,64.45,86.29,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0
8,HTMX11,5.964,21.2,5.1,24.0,-0.100,121.33,179.05,0.0,0.0,0.000,0.0,0.000,10.300,0.0,9.295,0.0
9,IVVB11,8.889,20.0,28.2,16.0,-0.077,257.48,417.20,0.0,0.0,0.000,0.0,0.000,10.900,0.0,9.915,0.0


In [462]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [463]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wss = wb.worksheet('Stockvar')
wsr = wb.worksheet('RealStatevar')
wssus = wb.worksheet('StockUSvar')
wseus = wb.worksheet('ETFUSvar')
wsport = wb.worksheet('Portfolio')

In [464]:
# Write data in the worksheets
wss.update([stockvar.columns.values.tolist()] + stockvar.values.tolist())
wsr.update([realstatevar.columns.values.tolist()] + realstatevar.values.tolist())
wssus.update([stockusvar.columns.values.tolist()] + stockusvar.values.tolist())
wseus.update([etfusvar.columns.values.tolist()] + etfusvar.values.tolist())
wsport.update([portfolio.columns.values.tolist()] + portfolio.values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'Portfolio!A1:Q19',
 'updatedRows': 19,
 'updatedColumns': 17,
 'updatedCells': 323}

In [465]:
# B3 REAL STATE FUNDS EXPLORER ROUTINE
#
# example tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MALL11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']
# Remove ".SA" from real state funds dataframe tickers
tickerradjusted = [nome.replace('.SA', '') for nome in tickerr]
# Remove "IFIX" ticker
if "IFIX" in tickerradjusted:
    tickerradjusted.remove("IFIX")
#

In [466]:
# Funds list filter, exclude tickers not in real state funds list
fundsexplorer = fundsexplorer[fundsexplorer['Fundos'].isin(tickerradjusted)]
# display(fundsexplorer)

In [467]:
# Open worksheet
wsrfunds = wb.worksheet('FundsExplorer')

In [468]:
# Write data in the worksheet
wsrfunds.update([fundsexplorer.columns.values.tolist()] + fundsexplorer.fillna('').values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'FundsExplorer!A1:AD21',
 'updatedRows': 21,
 'updatedColumns': 30,
 'updatedCells': 630}

In [469]:
# B3 STOCKS FUNDAMENTUS ROUTINE
#
# example tickers = ['^BVSP','ASAI3.SA','AURE3.SA','B3SA3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CGAS3.SA','CPFE3.SA','CSAN3.SA','EGIE3.SA','ELET6.SA','EMBR3.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','LEVE3.SA','LREN3.SA','ODPV3.SA','PETR4.SA','PRIO3.SA','STBP3.SA','TAEE11.SA','VALE3.SA','VIVT3.SA','WEGE3.SA']
# Remove ".SA" from stock dataframe tickers
tickersadjusted = [nome.replace('.SA', '') for nome in tickers]
# Remove "^BVSP" ticker
if "^BVSP" in tickersadjusted:
    tickersadjusted.remove("^BVSP")
#

In [470]:
# Fundamentus list filter, exclude tickers not in stocks list
fundamentus = fundamentus[fundamentus['Papel'].isin(tickersadjusted)]
fundamentus.columns = [col.replace('Papel', 'ticker') for col in fundamentus.columns]
fundamentus = fundamentus.sort_values(by='ticker')
# display(fundamentus)

In [471]:
# Open worksheet
wssfundamentus = wb.worksheet('Fundamentusold')

In [472]:
# Write data in the worksheet
wssfundamentus.update([fundamentus.columns.values.tolist()] + fundamentus.fillna('').values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'Fundamentusold!A1:U25',
 'updatedRows': 25,
 'updatedColumns': 21,
 'updatedCells': 525}

In [473]:
# US STOCKS YAHOO FINANCE ROUTINE
#
# Example tickersus = ['^GSPC','USDBRL=X','AAPL','AIG','BAC','RIO','DHI','EXC','KMB','KO','LOPE','LYB','MGA','MSFT','MSTR','NUE','NVDA','TGT','TMUS','UPS','UNH','XOM']
#
# Dicionário solicitado para armazenar os dados (cada valor é um dict por ticker)
yfstockusdata = {}

# Conjunto para acumular todas as chaves encontradas em todos os tickers
all_keys = set()

for t in tickersus:
    try:
        tk = yf.Ticker(t)
        info = tk.info or {}
        # Garante que o ticker esteja presente como campo
        info_row = {'Ticker': t}
        # adiciona todas as chaves/valores retornados por .info
        for k, v in info.items():
            info_row[k] = v
            all_keys.add(k)
        # armazena a linha
        yfstockusdata[t] = info_row
    except Exception as e:
        # Em caso de falha, registra o ticker com apenas o campo 'Ticker'
        # e mantém as demais chaves ausentes (serão NaN no DataFrame)
        yfstockusdata[t] = {'Ticker': t}
        # (opcional) você pode logar o erro se quiser:
        # print(f"Erro em {t}: {e}")

# Para garantir colunas ordenadas com 'Ticker' primeiro, montamos explicitamente a lista de colunas
cols_other = sorted(all_keys - {'Ticker'})  # ordena alfabeticamente as demais chaves (opcional)
cols_final = ['Ticker'] + cols_other

# Converte o dicionário para DataFrame mantendo índice numérico (0,1,2,...)
# Note: usamos list(yfstockusdata.values()) para preservar cada dict como uma linha
yfstockus = pd.DataFrame(list(yfstockusdata.values()))

# Reindexa as colunas para garantir 'Ticker' como primeira e as restantes presentes
# Existem casos em que algumas chaves não aparecem em nenhum ticker — isso não ocorrerá,
# pois all_keys foi montado a partir de .info de cada ticker.
# Ainda assim, para segurança:
existing_cols = [c for c in cols_final if c in yfstockus.columns]
yfstockus = yfstockus[existing_cols + [c for c in yfstockus.columns if c not in existing_cols]]

# Exibe o DataFrame (uso em Jupyter/IPython)
display(yfstockus)

,Ticker,52WeekChange,SandP52WeekChange,address1,address2,allTimeHigh,allTimeLow,ask,askSize,auditRisk,...,trailingPE,trailingPegRatio,triggerable,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,typeDisp,volume,website,zip
0,^GSPC,19.736935,NaN,NaN,NaN,6920.340000,4.400000,6881.6700,0,NaN,...,NaN,NaN,True,6111.516000,728.684100,0.119231,Index,3777566000,NaN,NaN
1,USDBRL=X,NaN,NaN,NaN,NaN,6.411100,1.525200,5.3772,0,NaN,...,NaN,NaN,True,5.589596,-0.214396,-0.038356,Currency,0,NaN,NaN
2,AAPL,0.217828,0.197369,One Apple Park Way,NaN,277.320000,0.049107,285.5800,1,7.0,...,36.242626,2.4662,True,223.302000,47.067993,0.210782,Equity,86167123,https://www.apple.com,95014
3,AIG,0.034456,0.197369,1271 Avenue of the Americas,NaN,2075.000000,6.600000,78.9200,3,6.0,...,14.758879,NaN,True,80.664650,-1.704651,-0.021133,Equity,2338560,https://www.aig.com,10020
4,BAC,0.293249,0.197369,Bank of America Corporate Center,100 North Tryon Street,55.080000,0.828125,53.5000,83,6.0,...,14.603825,1.2606,True,45.860750,7.589252,0.165485,Equity,39278767,https://www.bankofamerica.com,28255
5,RIO,0.103523,0.197369,6 St James’s Square,NaN,139.662500,7.500000,72.5300,2,NaN,...,11.423566,NaN,True,61.810000,9.929996,0.160654,Equity,7777899,https://www.riotinto.com,SW1Y 4AD
6,DHI,-0.126860,0.197369,1341 Horton Circle,NaN,199.850000,1.051366,149.2900,2,8.0,...,12.885048,1.8235,True,140.596800,8.483200,0.060337,Equity,2991760,https://www.drhorton.com,76011
7,EXC,0.214643,0.197369,10 South Dearborn Street,54th Floor PO Box 805398,65.713264,3.209700,48.4400,1,3.0,...,17.536121,2.0601,True,44.104300,2.015697,0.045703,Equity,8716383,https://www.exeloncorp.com,60680-5379
8,KMB,-0.104905,0.197369,PO Box 619100,NaN,160.160000,2.262105,119.7900,1,3.0,...,20.255499,4.8401,True,131.944660,-12.234657,-0.092726,Equity,3832896,https://www.kimberly-clark.com,75261-9100
9,KO,0.058209,0.197369,One Coca-Cola Plaza,NaN,74.380000,0.182292,68.8900,23,2.0,...,22.814570,2.2073,True,69.205150,-0.305145,-0.004409,Equity,11611988,https://www.coca-colacompany.com,30313


In [474]:
# Open worksheet
# wssyfstockus = wb.worksheet('YFStockUS')

In [475]:
# Write data in the worksheet
# wssyfstockus.update([yfstockus.columns.values.tolist()] + yfstockus.fillna('').values.tolist())

In [476]:
# --- Função de sanitização por valor ---
def sanitize_value(v, maxlen=500):
    # None / NaN
    if v is None:
        return ''
    # floats: rejeitar NaN/Inf
    if isinstance(v, float):
        if not np.isfinite(v):
            return ''
        return float(v)
    # números inteiros e booleanos são OK
    if isinstance(v, (int, bool, np.integer, np.bool_)):
        return int(v) if isinstance(v, (int, np.integer)) else bool(v)
    # strings curtas OK (trunca se muito longas)
    if isinstance(v, str):
        return v if len(v) <= maxlen else v[:maxlen]
    # séries temporais / numpy types convertidos para string truncada
    # para todos os demais tipos (dict, list, Timestamp, ndarray, Decimal, etc.)
    try:
        s = str(v)
        return s if len(s) <= maxlen else s[:maxlen]
    except Exception:
        return ''

# --- Sanitizar coluna a coluna usando Series.map (evita applymap) ---
yfstockus_sanitized = yfstockus.copy()

for col in yfstockus_sanitized.columns:
    # Usa map para aplicar a função de maneira vetorizada por coluna
    yfstockus_sanitized[col] = yfstockus_sanitized[col].map(lambda x: sanitize_value(x))

# --- Monta as linhas para enviar ao Google Sheets ---
rows = [yfstockus_sanitized.columns.tolist()] + yfstockus_sanitized.values.tolist()

# --- Abre/Cria worksheet e atualiza ---
try:
    wssyfstockus = wb.worksheet('YFStockUS')
except Exception as e:
    # se não existir, cria (ajuste linhas/cols se quiser um tamanho específico)
    wssyfstockus = wb.add_worksheet(title='YFStockUS', rows=str(len(rows)+10), cols=str(len(rows[0])+5))

# Faz o update (agora com dados serializáveis)
wssyfstockus.update(rows)


{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'YFStockUS!A1:GE23',
 'updatedRows': 23,
 'updatedColumns': 187,
 'updatedCells': 4301}